In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv")
data.head()

In [ ]:
data.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
data.head()

In [ ]:
data['v1'].unique()

In [ ]:
data=data.rename(columns = {"v1" : "target", "v2" : "sms"})

In [ ]:
data.head()

In [ ]:
data['lenght']=data['sms'].str.len()

In [ ]:
data.head()

In [ ]:
y=pd.get_dummies(data['target'], drop_first = True)
data=pd.concat([data,y],axis=1)
data.drop("target",axis=1,inplace=True)

In [ ]:
def meassage_lenght(msg):
    msg_words=msg.split(" ")
    len_msg=len(msg_words)
    
    return len_msg

In [ ]:
print(meassage_lenght(data.iloc[1][0]))

In [ ]:
data['word_count']=data['sms'].apply(meassage_lenght)

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.hist(data['word_count'],bins=50,alpha=0.5)
plt.hist(data['lenght'],bins=50,alpha=0.5)
plt.xlabel("word lenght")
plt.ylabel("Group lenght")
plt.title("word Lenght histgoramm")

In [ ]:
data1=data[(data['word_count']>80) & (data['lenght']>100)]

In [ ]:
data1

In [ ]:
import string
string.punctuation

In [ ]:
def remove_punctuation(text):
    new_text="".join([char for char in text if char not in string.punctuation])
    return new_text

In [ ]:
data['new_sms']=data['sms'].apply(lambda row:remove_punctuation(row))

In [ ]:
data.head()

In [ ]:
import re
def tockenize(text):
    tokens=re.split('\W+',text)
    return tokens

In [ ]:
data['tokenized_row']=data['new_sms'].apply(lambda row:tockenize(row.lower()))

In [ ]:
data.head()

In [ ]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')
stopwords[:5]

In [ ]:
def revomestopword(text):
    clean_text=[char for char in text if char not in stopwords]
    return clean_text

In [ ]:
data['clean_text']=data['tokenized_row'].apply(lambda x: revomestopword(x))

In [ ]:
data.head()

In [ ]:
import nltk
from nltk.stem.porter import *
ps=PorterStemmer()
from nltk import PorterStemmer
def stemming(tokenized_row):
    stemmed_text=[ps.stem(char) for char in tokenized_row]
    return stemmed_text

In [ ]:
data['stemmed_text']=data['clean_text'].apply(lambda char :stemming(char))

In [ ]:
data[['sms','stemmed_text']].head()

In [ ]:
def final_text(stemmed_text):
    final_text=" ".join(char for char in stemmed_text)
    return final_text

In [ ]:
data['final_text']=data['stemmed_text'].apply(lambda x:final_text(x))

In [ ]:
data.head()

In [ ]:
df=data.drop(['new_sms','tokenized_row','clean_text','stemmed_text','word_count'],axis=1)

In [ ]:
df.head()

In [ ]:
x=df.drop(['spam'],axis=1)
y=df['spam']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

# Count Vectorization
It involves counting the number of occurrences of each word/token in a given text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
cv=CountVectorizer(max_features=500)
temp_train=cv.fit_transform(x_train['final_text']).toarray()
temp_test=cv.transform(x_test['final_text']).toarray()

In [ ]:
tf=TfidfTransformer()
temp_train=tf.fit_transform(temp_train)
temp_test=tf.transform(temp_test)

In [ ]:
temp_train=pd.DataFrame(temp_train.toarray(),index=x_train.index)
temp_test=pd.DataFrame(temp_test.toarray(),index=x_test.index)
x_train=pd.concat([x_train,temp_train],axis=1,sort=False)
x_test=pd.concat([x_test,temp_test],axis=1,sort=False)

In [ ]:
x_train.head()

In [ ]:
x_train.drop(['sms','final_text'],axis=1,inplace=True)
x_test.drop(['sms','final_text'],axis=1,inplace=True)
x_train.head()

# Model Train

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Multinomial Naive Bayes

In [ ]:
model=MultinomialNB()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("Accuracy_score:",accuracy_score(y_test,y_pred))

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("Accuracy_Score:",accuracy_score(y_test,y_pred))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("Accuracy_score:",accuracy_score(y_test,y_pred))